In [11]:
import tensorflow as tf
import os

def echo_tensor(sess, tensor, prefix = ''):
    # 注意： print() 显示时会把元素之间的逗号去掉
    if( isinstance(tensor, tf.Tensor) ):
        # print( '{0} tensor.shape = {1}, tensor = {2}\r\n'.format(prefix, sess.run(tf.shape(tensor)), sess.run(tensor)) )
        print( '{0} tensor = {1}\r\n'.format(prefix, sess.run(tensor)) )
    else:
        print( '{0} not_tensor = {1}\r\n'.format(prefix, tensor) )

# 从csv文件读取数据，加载解析，创建批次读取张量多行数据
# 调用举例: read_csv(4, ['1.csv', '2.csv'], [[0], [0.], ['']])
# 
# 如果 csv 文件中数据为：
# 1,2,3,4
# 11,12,13,14
# 21,22,23,24
# 31,32,33,34
# 41,42,43,44
# 51,52,53,54
# 
# 那么输出会是：
# [[21, 51, 1, 61], [22, 52, 2, 62], [23, 53, 3, 63], [24, 54, 4, 64]]
# 
def read_csv(batch_size, file_name, record_defaults):
    file_path = list( map(lambda name: os.path.join(os.getcwd(), name), file_name) )
    print('file_path = ', file_path, '\r\n')
    file_queue = tf.train.string_input_producer(file_path) 

    reader = tf.TextLineReader(skip_header_lines = 1)

    key, value = reader.read(file_queue)
    print(key, value)

    decoded = tf.decode_csv(value, record_defaults, ',')
    return tf.train.shuffle_batch(decoded, batch_size=batch_size, capacity=batch_size * 50, min_after_dequeue=batch_size) # 读取文件，加载张量batch_size行 


def inputs(): # 读取或生成训练数据 X 及期望输出 Y
    passenger_id, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked = \
    read_csv(10, ['Titanic_train.csv'], [[0.0], [0.0], [0], [''], [''], [0.0], [0.0], [0.0], [''], [0.0], [''], ['']])
    
    is_first_class = tf.to_float(tf.equal(pclass, [1]))
    is_second_class = tf.to_float(tf.equal(pclass, [2]))
    is_third_class = tf.to_float(tf.equal(pclass, [3]))
    
    gender = tf.to_float(tf.equal(sex, ['female']))
    
    features = tf.transpose(tf.pack([is_first_class, is_second_class, is_third_class, gender, age]))
    survived = tf.reshape(survived, [10, 1])
    return features, survived

with tf.Session() as sess:
    print( 'Session: start' )
    tf.initialize_all_variables().run() 
    
    # data_rows = read_csv(1, 'iris.csv', [[''],[''],[''],[''],['']])  
    X, Y = inputs()

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    # echo_tensor( sess, threads, 'threads_1' )
    
    
    try:
    
        for i in range(2):
            # print(sess.run(data_rows))
            # print(sess.run(X))
            echo_tensor(sess, X, 'X_' + str(i))

    except tf.errors.OutOfRangeError:
        print( 'Done training -- epoch limit reached' )

    finally:
        # When done, ask the threads to stop.
        # echo_tensor( sess, threads, 'threads_2' )
        coord.request_stop()        
        coord.join(threads)

    sess.close()
    print( 'Session: close' )

Session: start
file_path =  ['/home/yangyuqi/work/tf_algorithm_example/Titanic_train.csv'] 

Tensor("ReaderRead_10:0", shape=(), dtype=string) Tensor("ReaderRead_10:1", shape=(), dtype=string)
X_0 tensor = [[  1.   0.   0.   1.  38.]
 [  0.   0.   1.   1.  26.]
 [  0.   0.   1.   1.   4.]
 [  0.   0.   1.   0.  39.]
 [  0.   0.   1.   0.  35.]
 [  1.   0.   0.   1.  35.]
 [  1.   0.   0.   1.  58.]
 [  0.   1.   0.   1.  14.]
 [  1.   0.   0.   0.  54.]
 [  0.   1.   0.   0.   0.]]

X_1 tensor = [[  0.   0.   1.   0.   2.]
 [  0.   1.   0.   0.  35.]
 [  0.   0.   1.   0.   2.]
 [  1.   0.   0.   0.  28.]
 [  0.   0.   1.   1.   8.]
 [  0.   0.   1.   0.  20.]
 [  0.   0.   1.   1.  38.]
 [  0.   1.   0.   0.  34.]
 [  0.   0.   1.   1.  31.]
 [  0.   0.   1.   1.   0.]]

Session: close
